In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier, plot_tree,export_graphviz
from sklearn import tree
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

### Lendo a base de dados


In [5]:
dados = pd.read_excel('CCARDBALANCE.xlsx')
dados = dados.drop('ID',axis = 1)
display(dados)

,Renda,Limite,Rating,Cartoes,Idade,Educacao,Sexo,Estudante,Casado,Etnia,Gasto Mensal
0,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331
...,...,...,...,...,...,...,...,...,...,...,...
395,12.096,4100,307,3,32,13,Male,No,Yes,Caucasian,560
396,13.364,3838,296,5,65,17,Male,No,No,African American,480
397,57.872,4171,321,5,67,12,Female,No,Yes,Caucasian,138
398,37.728,2525,192,1,44,13,Male,No,Yes,Caucasian,0


### Corrigindo os valores de string para numeric

In [6]:
dados['Estudante'] = dados['Estudante'].replace({'Yes': 1, 'No': 0})
dados['Casado'] = dados['Casado'].replace({'Yes': 1, 'No': 0})
dados['Etnia'] = dados['Etnia'].replace({'Caucasian': 0, 'Asian': 1,'African American' : 2})
dados['Sexo'] = dados['Sexo'].replace({' Male':1,'Female':0})

In [7]:
display(dados)

,Renda,Limite,Rating,Cartoes,Idade,Educacao,Sexo,Estudante,Casado,Etnia,Gasto Mensal
0,14.891,3606,283,2,34,11,1,0,1,0,333
1,106.025,6645,483,3,82,15,0,1,1,1,903
2,104.593,7075,514,4,71,11,1,0,0,1,580
3,148.924,9504,681,3,36,11,0,0,0,1,964
4,55.882,4897,357,2,68,16,1,0,1,0,331
...,...,...,...,...,...,...,...,...,...,...,...
395,12.096,4100,307,3,32,13,1,0,1,0,560
396,13.364,3838,296,5,65,17,1,0,0,2,480
397,57.872,4171,321,5,67,12,0,0,1,0,138
398,37.728,2525,192,1,44,13,1,0,1,0,0


### Usando o primeiro modelo, Arvore de decisão, separando as variaveis X do target y, o target é a variavel que a gente quer prever

In [8]:
X = dados.drop('Gasto Mensal', axis = 1)
y = dados['Gasto Mensal']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=0)
modelo = DecisionTreeRegressor()
modelo.fit(X_train,y_train)
y_pred = modelo.predict(X_test)

### Não sei porque o R^2 deu tão alto, na minha cabeça ele variava de 0 a 1 

In [9]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE:', rmse)

RMSE: 142.3166539797785


# Teste

In [69]:
teste = {'Renda':13.200 ,'Limite':800,'Rating':600,'Cartoes':1,'Idade':20,'Educacao':17,'Sexo':1,'Estudante':1,'Casado':0,'Etnia':1}
teste_df = pd.DataFrame(data = teste, index=[0])
display(teste_df)
previsao = modelo.predict(teste_df)
print(previsao)

,Renda,Limite,Rating,Cartoes,Idade,Educacao,Sexo,Estudante,Casado,Etnia
0,30.0,1000,750,1,23,17,1,1,0,1


[1216.]


### Quão importante é cada característica para o quanto de gasto mensal a pessoa vai ter, aqui podemos ver que etnia não faz a menor diferença.

In [62]:
# Importância das características
importancias = modelo.feature_importances_
nomes_features = X.columns

# Imprimir importância das características
for feature, importancia in zip(nomes_features, importancias):
    print(feature, importancia)
print("Acurácia: ", modelo.score(X, y))

Renda 0.08122318224923296
Limite 0.26131801162132146
Rating 0.5854522832294905
Cartoes 0.002886399261461944
Idade 0.00784504587041282
Educacao 0.004145978853957303
Sexo 0.0005599723663750662
Estudante 0.05251030739099163
Casado 0.0018333813028423769
Etnia 0.002225437853914011
Acurácia:  0.9846272830224966


### Pra ver o grafico que o professor fez, tenho que cortar ele um pouco ainda.

In [ ]:
#Numero de folhas
#print("Número de Folhas: ", modelo.get_n_leaves())

#Profundidade
#print("Profundidade: ",modelo.get_depth())

#Obtem os parametros repassados. Nesse caso, por exemplo

 
#plotando a arvore de decisão com a função padrão do sklearn
#plt.figure( figsize=(7,7),dpi=250)
#plot_tree(modelo,feature_names=['Renda', 'Limite', 'Rating', 'Cartoes', 'Idade', 'Educacao', 'Sexo',
#       'Estudante', 'Casado', 'Etnia'],  
#                      class_names=['Gasto Mensal'],
#                      filled=True, rounded=True,fontsize=5.0
#                   )
#plt.show()

### Utilizando RandomForest

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=0)
modelo = RandomForestRegressor()
modelo.fit(X_train,y_train)
y_pred = modelo.predict(X_test)

In [71]:
resultado = modelo.predict(teste_df)
print(resultado)

[1073.85]


### R^2 menor, eu tambem não sei porque

In [30]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE:', rmse)

RMSE: 90.3015022577144


### Features do RandomForest são mais aprimoradas e a acurácia esta bem mais alta, o que pode ser ruim já que eu não tenho ideia se eu treinei o modelo do jeito certo.

In [63]:
# Importância das características
importancias = modelo.feature_importances_
nomes_features = X.columns

# Imprimir importância das características
for feature, importancia in zip(nomes_features, importancias):
    print(feature, importancia)
print("Acurácia: ", modelo.score(X, y))

Renda 0.08122318224923296
Limite 0.26131801162132146
Rating 0.5854522832294905
Cartoes 0.002886399261461944
Idade 0.00784504587041282
Educacao 0.004145978853957303
Sexo 0.0005599723663750662
Estudante 0.05251030739099163
Casado 0.0018333813028423769
Etnia 0.002225437853914011
Acurácia:  0.9846272830224966


### Usando XGBoost.

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=0)
modelo = XGBRegressor()
modelo.fit(X_train,y_train)
y_pred = modelo.predict(X_test)

In [74]:
resultado = modelo.predict(teste_df)
print(resultado)

[795.0564]


In [33]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE:', rmse)

RMSE: 78.7833349922023


### Acurácia de quase 100%.

In [66]:
# Importância das características
importancias = modelo.feature_importances_
nomes_features = X.columns

# Imprimir importância das características
for feature, importancia in zip(nomes_features, importancias):
    print(feature, importancia)
print("Acurácia: ", modelo.score(X, y))

Renda 0.026520869
Limite 0.09002677
Rating 0.66081244
Cartoes 0.0019851765
Idade 0.002206139
Educacao 0.0015670513
Sexo 0.001480999
Estudante 0.21272369
Casado 0.001795675
Etnia 0.00088117406
Acurácia:  0.9926405812620638
